# Otsu法による自動閾値決定

このノートブックでは、otsu法による自動閾値決定について学びます。

## 目次
1. [Otsu法](#otsu法)
2. [ヒストグラム解析](#ヒストグラム解析)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from skimage import filters, feature

plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["axes.unicode_minus"] = False
np.random.seed(42)
print("ライブラリのインポートが完了しました。")

## Otsu法

In [ ]:
# Otsu法による自動閾値決定
# ヒストグラムのクラス間分散を最大化する閾値を自動的に決定

# サンプル画像の作成（2つの輝度レベル）
img = np.zeros((200, 200), dtype=np.uint8)
img[50:150, 50:150] = 200  # 明るい領域
img = img + np.random.randint(-30, 30, (200, 200), dtype=np.int16)
img = np.clip(img, 0, 255).astype(np.uint8)

# Otsu法による閾値決定
ret, thresh_otsu = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# 固定閾値との比較
ret_fixed, thresh_fixed = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

# 結果の表示
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(img, cmap='gray')
axes[0, 0].set_title('元の画像')
axes[0, 0].axis('off')

axes[0, 1].imshow(thresh_fixed, cmap='gray')
axes[0, 1].set_title(f'固定閾値 (127)')
axes[0, 1].axis('off')

axes[1, 0].imshow(thresh_otsu, cmap='gray')
axes[1, 0].set_title(f'Otsu法 (閾値: {ret:.0f})')
axes[1, 0].axis('off')

# ヒストグラムの表示
axes[1, 1].hist(img.flatten(), bins=256, range=(0, 256), color='black', alpha=0.7)
axes[1, 1].axvline(ret, color='red', linestyle='--', linewidth=2, label=f'Otsu閾値: {ret:.0f}')
axes[1, 1].axvline(127, color='blue', linestyle='--', linewidth=2, label='固定閾値: 127')
axes[1, 1].set_xlabel('Pixel Value')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('ヒストグラムと閾値')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Otsu法で決定された閾値: {ret:.0f}")
print("Otsu法は、クラス間分散を最大化する閾値を自動的に決定します。")

## ヒストグラム解析

In [ ]:
# ヒストグラム解析による閾値決定の理解

# より複雑な画像でOtsu法を適用
img_complex = np.random.randint(0, 256, (200, 200), dtype=np.uint8)
# 2つの分布を作成
img_complex[30:100, 30:100] = np.random.randint(180, 255, (70, 70), dtype=np.uint8)
img_complex[100:170, 100:170] = np.random.randint(0, 80, (70, 70), dtype=np.uint8)

# Otsu法を適用
ret2, thresh2 = cv2.threshold(img_complex, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# ヒストグラムの詳細表示
hist, bins = np.histogram(img_complex.flatten(), bins=256, range=(0, 256))

# 結果の表示
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(img_complex, cmap='gray')
axes[0, 0].set_title('複雑な画像')
axes[0, 0].axis('off')

axes[0, 1].imshow(thresh2, cmap='gray')
axes[0, 1].set_title(f'Otsu法 (閾値: {ret2:.0f})')
axes[0, 1].axis('off')

# ヒストグラム
axes[1, 0].bar(bins[:-1], hist, width=1, color='black', alpha=0.7)
axes[1, 0].axvline(ret2, color='red', linestyle='--', linewidth=2, label=f'Otsu閾値: {ret2:.0f}')
axes[1, 0].set_xlabel('Pixel Value')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('ヒストグラム')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# クラス間分散の計算（理解のため）
thresholds = np.arange(0, 256)
variances = []
for t in thresholds:
    w0 = np.sum(hist[:t]) / np.sum(hist)
    w1 = 1 - w0
    if w0 > 0 and w1 > 0:
        m0 = np.sum(bins[:-1][:t] * hist[:t]) / np.sum(hist[:t]) if np.sum(hist[:t]) > 0 else 0
        m1 = np.sum(bins[:-1][t:] * hist[t:]) / np.sum(hist[t:]) if np.sum(hist[t:]) > 0 else 0
        variance = w0 * w1 * (m0 - m1)**2
        variances.append(variance)
    else:
        variances.append(0)

axes[1, 1].plot(thresholds, variances, color='blue', linewidth=2)
axes[1, 1].axvline(ret2, color='red', linestyle='--', linewidth=2, label=f'最適閾値: {ret2:.0f}')
axes[1, 1].set_xlabel('Threshold')
axes[1, 1].set_ylabel('Between-class Variance')
axes[1, 1].set_title('クラス間分散')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nOtsu法の原理:")
print("クラス間分散を最大化する閾値を選択することで、")
print("2つのクラス（前景と背景）を最も良く分離できます。")